In [53]:
import pandas as pd

df = pd.read_csv('/content/orgsexcellist.csv')

In [54]:
df.head()

,Organization Name,POA,Org Phone,Org City,Org State,Representative,Rep City,Rep State,Rep Zip,Registration Num
0,African American PTSD Association,091,253-589-0766,Lakewood,WA,"Anderson, Edgar B",Hazel Crest,IL,60429,8239.0
1,African American PTSD Association,091,253-589-0766,Lakewood,WA,"Anderson, Victoria C",Country Club Hills,IL,60478,24028.0
2,African American PTSD Association,091,253-589-0766,Lakewood,WA,"Berry, Joe E",NaN,NaN,NaN,53299.0
3,African American PTSD Association,091,253-589-0766,Lakewood,WA,"Blue, James T",Chicago,IL,60636,8240.0
4,African American PTSD Association,091,253-589-0766,Lakewood,WA,"Buchanan, Mosea",Country Clubs Hills,IL,60478,32966.0


In [55]:
df['Address'] = None
df['Website'] = None
df['Rating'] = None
df['Coordinates (lat long)'] = None

In [16]:
df.head()

,Organization Name,POA,Org Phone,Org City,Org State,Representative,Rep City,Rep State,Rep Zip,Registration Num,Address,Website,Rating,Coordinates (lat long)
0,African American PTSD Association,091,253-589-0766,Lakewood,WA,"Anderson, Edgar B",Hazel Crest,IL,60429,8239.0,None,None,None,None
1,African American PTSD Association,091,253-589-0766,Lakewood,WA,"Anderson, Victoria C",Country Club Hills,IL,60478,24028.0,None,None,None,None
2,African American PTSD Association,091,253-589-0766,Lakewood,WA,"Berry, Joe E",NaN,NaN,NaN,53299.0,None,None,None,None
3,African American PTSD Association,091,253-589-0766,Lakewood,WA,"Blue, James T",Chicago,IL,60636,8240.0,None,None,None,None
4,African American PTSD Association,091,253-589-0766,Lakewood,WA,"Buchanan, Mosea",Country Clubs Hills,IL,60478,32966.0,None,None,None,None


In [38]:
# Single Request

import http.client
import json

conn = http.client.HTTPSConnection("google.serper.dev")
payload = json.dumps({
  "q": "African American PTSD Association	Lakewood"
})
headers = {
  'X-API-KEY': 'd7e0f8413e0e24e12baec8d766992e1d17d05dc5',
  'Content-Type': 'application/json'
}
conn.request("POST", "/places", payload, headers)
res = conn.getresponse()
data = res.read()
freshData = json.loads(data.decode("utf-8"))
freshData['places'][0]

{"searchParameters":{"q":"African American PTSD Association\tLakewood","type":"places","engine":"google"},"places":[{"position":1,"title":"African American PTSD Associates","address":"9129 Veterans Dr SW, Lakewood, WA 98498","latitude":47.143884,"longitude":-122.560005,"rating":4.1,"ratingCount":18,"category":"Veterans organization","phoneNumber":"(253) 589-0766","website":"https://aaptsda.org/","cid":"3268232693651549922"}]}


In [56]:
# Multiple requests
def fetch_data(org_name, org_city):
    conn = http.client.HTTPSConnection("google.serper.dev")
    query = f"{org_name} {org_city}"
    payload = json.dumps({
        "q": query
    })
    headers = {
        'X-API-KEY': 'd7e0f8413e0e24e12baec8d766992e1d17d05dc5',
        'Content-Type': 'application/json'
    }
    conn.request("POST", "/places", payload, headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))

In [58]:
unique_orgs = df[['Organization Name', 'Org City']].drop_duplicates()

In [59]:
for index, row in unique_orgs.iterrows():
    org_name = row['Organization Name']
    org_city = row['Org City']

    api_response = fetch_data(org_name, org_city)

    if 'places' in api_response and len(api_response['places']) > 0:
        place_info = api_response['places'][0]
        address = place_info.get('address')
        website = place_info.get('website')
        rating = place_info.get('rating')
        coordinates = f"{place_info.get('latitude')}, {place_info.get('longitude')}"

        # Update all rows with the same Organization Name and Org City
        df.loc[(df['Organization Name'] == org_name) & (df['Org City'] == org_city), 'Address'] = address
        df.loc[(df['Organization Name'] == org_name) & (df['Org City'] == org_city), 'Website'] = website
        df.loc[(df['Organization Name'] == org_name) & (df['Org City'] == org_city), 'Rating'] = rating
        df.loc[(df['Organization Name'] == org_name) & (df['Org City'] == org_city), 'Coordinates (lat long)'] = coordinates

In [124]:
df.isna().sum()

,0
orgName,0
poa,0
orgPhone,2041
orgCity,0
orgState,0
repName,2
repCity,276
repState,260
repZip,326
registrationNum,2


In [74]:
pd.set_option('display.max_rows', None)
df

Output hidden; open in https://colab.research.google.com to view.

In [63]:
specific_rating = 4.4
specific_lat_long = "41.14821250000001, -104.7851649"

# Update the relevant rows in the DataFrame
df.loc[
    (df['Organization Name'] == 'Wyoming Veterans Commission') &
    (df['Org City'] == 'Cheyenne'),
    ['Rating', 'Coordinates (lat long)']
] = specific_rating, specific_lat_long

In [66]:
# Define the specific values from the screenshot
specific_address = "200 Maryland Ave NE lower level, Washington, DC 20002"
specific_website = "https://www.vfw.org/"
specific_rating = 4.6
specific_lat_long = "38.8923785, -77.00308749999999"

# Update the relevant rows in the DataFrame
df.loc[
    (df['Organization Name'] == 'Veterans of Foreign Wars') &
    (df['Org City'] == 'Washington'),
    ['Address', 'Website', 'Rating', 'Coordinates (lat long)']
] = specific_address, specific_website, specific_rating, specific_lat_long

In [72]:
# Define the specific values from the screenshot and the correct website
specific_address = "1599 Dayton St, Aurora, CO 80010"
specific_website = "https://www.trea.org/"
specific_rating = 4.3
specific_lat_long = "39.741792, -104.875439"

# Update the relevant rows in the DataFrame
df.loc[
    (df['Organization Name'] == 'The Retired Enlisted Association') &
    (df['Org City'] == 'Centennial'),
    ['Address', 'Website', 'Rating', 'Coordinates (lat long)']
] = specific_address, specific_website, specific_rating, specific_lat_long

In [88]:
specific_website = "https://veterans.vi.gov/"

# Update the relevant rows in the DataFrame
df.loc[
    df['Organization Name'] == 'Virgin Islands Office of Veterans Affairs',
    'Website'
] = specific_website

In [96]:
# Define the specific phone number
specific_phone_number = "(301) 585-4000"

# Update the relevant rows in the DataFrame
df.loc[
    df['Organization Name'] == "Veterans' Voice of America",
    'Org Phone'
] = specific_phone_number

In [97]:
df[df['Org Phone'].isnull()]

,Organization Name,POA,Org Phone,Org City,Org State,Representative,Rep City,Rep State,Rep Zip,Registration Num,Address,Website,Rating,Coordinates (lat long)
9088,National Association of County Veterans Servic...,064,NaN,Washington,DC,"Abbott, Alice M",Decorah,IA,52101,25332.0,"25 Massachusetts Ave NW #500, Washington, DC 2...",http://nacvso.org/,None,"38.8983777, -77.0103683"
9089,National Association of County Veterans Servic...,064,NaN,Washington,DC,"Abel, Jami M",Painesville,OH,44077-0490,102.0,"25 Massachusetts Ave NW #500, Washington, DC 2...",http://nacvso.org/,None,"38.8983777, -77.0103683"
9090,National Association of County Veterans Servic...,064,NaN,Washington,DC,"Abraham, Sonia R",Port Arthur,TX,77640,52942.0,"25 Massachusetts Ave NW #500, Washington, DC 2...",http://nacvso.org/,None,"38.8983777, -77.0103683"
9091,National Association of County Veterans Servic...,064,NaN,Washington,DC,"Ackler, Joshua G",Lisbon,OH,44432,55360.0,"25 Massachusetts Ave NW #500, Washington, DC 2...",http://nacvso.org/,None,"38.8983777, -77.0103683"
9092,National Association of County Veterans Servic...,064,NaN,Washington,DC,"Adair, Randy M",Antigo,WI,54409,51706.0,"25 Massachusetts Ave NW #500, Washington, DC 2...",http://nacvso.org/,None,"38.8983777, -77.0103683"
9093,National Association of County Veterans Servic...,064,NaN,Washington,DC,"Adams, Maureen S",St. Joseph,MI,49085,41542.0,"25 Massachusetts Ave NW #500, Washington, DC 2...",http://nacvso.org/,None,"38.8983777, -77.0103683"
9094,National Association of County Veterans Servic...,064,NaN,Washington,DC,"Adekinle, Gabriel A",Minneapolis,MN,55487,55354.0,"25 Massachusetts Ave NW #500, Washington, DC 2...",http://nacvso.org/,None,"38.8983777, -77.0103683"
9095,National Association of County Veterans Servic...,064,NaN,Washington,DC,"Adkisson, Dennis L",Mt. Ayv,IA,50854,32039.0,"25 Massachusetts Ave NW #500, Washington, DC 2...",http://nacvso.org/,None,"38.8983777, -77.0103683"
9096,National Association of County Veterans Servic...,064,NaN,Washington,DC,"Agan, Nicholas R",Dubuque,IA,52001,39578.0,"25 Massachusetts Ave NW #500, Washington, DC 2...",http://nacvso.org/,None,"38.8983777, -77.0103683"
9097,National Association of County Veterans Servic...,064,NaN,Washington,DC,"Ahlich, Ross M",Midland,MI,48640,14003.0,"25 Massachusetts Ave NW #500, Washington, DC 2...",http://nacvso.org/,None,"38.8983777, -77.0103683"


In [98]:
import pandas as pd

# Assuming your DataFrame is named df

# Function to convert column names to camelCase
def to_camel_case(snake_str):
    components = snake_str.split(' ')
    return components[0].lower() + ''.join(x.title() for x in components[1:])

# Apply the function to each column name
df.columns = [to_camel_case(col) for col in df.columns]

In [100]:
df = df.rename(columns={
    'organizationName': 'orgName',
    'representative': 'repName'
})

In [130]:
# Define the specific values to update
new_address = "2425 Margaret St N, North Saint Paul, MN 55109"
new_city = "North Saint Paul"
new_state = "MN"
new_phone = None  # Null value
new_rating = None  # Null value
new_coordinates = '44.954445, -93.091301'  # Null value
new_website = "www.veteransvoice.org"

# Update the relevant rows in the DataFrame
df.loc[df['orgName'] == "Veterans' Voice of America", ['address', 'orgCity', 'orgState', 'orgPhone', 'rating', 'coordinates(LatLong)', 'website']] = [
    new_address, new_city, new_state, new_phone, new_rating, new_coordinates, new_website
]

In [117]:
df[df['orgName'] == "Veterans' Voice of America"]

,orgName,poa,orgPhone,orgCity,orgState,repName,repCity,repState,repZip,registrationNum,address,website,rating,coordinates(LatLong)
16181,Veterans' Voice of America,J3C,None,North Saint Paul,MN,"Bouy, Kenneth C",Silver Spring,MD,20910,45000.0,"2425 Margaret St N, North Saint Paul, MN 55109",www.veteransvoice.org,None,"44.954445, -93.091301"
16182,Veterans' Voice of America,J3C,None,North Saint Paul,MN,"Buza, Dana E",Detroit,MI,48226,50444.0,"2425 Margaret St N, North Saint Paul, MN 55109",www.veteransvoice.org,None,"44.954445, -93.091301"
16183,Veterans' Voice of America,J3C,None,North Saint Paul,MN,"Chaney, Elaine L",Silver Spring,MD,20910,1197.0,"2425 Margaret St N, North Saint Paul, MN 55109",www.veteransvoice.org,None,"44.954445, -93.091301"
16184,Veterans' Voice of America,J3C,None,North Saint Paul,MN,"Flesch, Paula A",Silver Spring,MD,20910,54404.0,"2425 Margaret St N, North Saint Paul, MN 55109",www.veteransvoice.org,None,"44.954445, -93.091301"
16185,Veterans' Voice of America,J3C,None,North Saint Paul,MN,"Greene, Larry C",Silver Spring,MD,20910,19199.0,"2425 Margaret St N, North Saint Paul, MN 55109",www.veteransvoice.org,None,"44.954445, -93.091301"
16186,Veterans' Voice of America,J3C,None,North Saint Paul,MN,"Harris, Patricia A",Silver Springs,MD,20910,53774.0,"2425 Margaret St N, North Saint Paul, MN 55109",www.veteransvoice.org,None,"44.954445, -93.091301"
16187,Veterans' Voice of America,J3C,None,North Saint Paul,MN,"Harris, Yolonda C",Silver Spring,MD,20910,46706.0,"2425 Margaret St N, North Saint Paul, MN 55109",www.veteransvoice.org,None,"44.954445, -93.091301"
16188,Veterans' Voice of America,J3C,None,North Saint Paul,MN,"Lichy, Gary R",Silver Spring,MD,20910,38262.0,"2425 Margaret St N, North Saint Paul, MN 55109",www.veteransvoice.org,None,"44.954445, -93.091301"
16189,Veterans' Voice of America,J3C,None,North Saint Paul,MN,"McCabe, Marc L",Bay Pines,FL,33744,16592.0,"2425 Margaret St N, North Saint Paul, MN 55109",www.veteransvoice.org,None,"44.954445, -93.091301"
16190,Veterans' Voice of America,J3C,None,North Saint Paul,MN,"Nedd, Angela D",Arlington,VA,22209,55561.0,"2425 Margaret St N, North Saint Paul, MN 55109",www.veteransvoice.org,None,"44.954445, -93.091301"


In [ ]:
orgExtendedList

In [118]:
df.to_csv('orgExtendedList.csv')

In [133]:
import pandas as pd

# Select the required columns for df2
df2 = df[['orgName', 'address', 'orgCity', 'orgState', 'orgPhone', 'website', 'repName', 'rating', 'coordinates(LatLong)']]

# Group by orgName and other fields, aggregate representatives into a list, and take first non-null values for others
df2_grouped = df2.groupby(['orgName', 'address', 'orgCity', 'orgState', 'website'], as_index=False).agg({
    'repName': lambda x: list(x.dropna().unique()) if not x.isnull().all() else [],  # Empty list if no repNames
    'orgPhone': 'first',  # Take the first non-null orgPhone
    'rating': 'first',    # Take the first non-null rating
    'coordinates(LatLong)': 'first'  # Take the first non-null coordinates
})

# Display the concise DataFrame
print(df2_grouped)

                                              orgName  \
0                   African American PTSD Association   
1              Alabama Department of Veterans Affairs   
2                                     American Legion   
3                                  American Red Cross   
4                                   American Veterans   
5             Arizona Department of Veterans Services   
6             Arkansas Department of Veterans Affairs   
7                   Armed Forces Services Corporation   
8                        Blinded Veterans Association   
9          California Department of Veterans Services   
10                   Catholic War Veterans of the USA   
11              Colorado Division of Veterans Affairs   
12         Connecticut Department of Veterans Affairs   
13                 Dale K. Graham Veterans Foundation   
14            Delaware Commission of Veterans Affairs   
15                         Disabled American Veterans   
16                          Fle

In [134]:
df2_grouped

,orgName,address,orgCity,orgState,website,repName,orgPhone,rating,coordinates(LatLong)
0,African American PTSD Association,"9129 Veterans Dr SW, Lakewood, WA 98498",Lakewood,WA,https://aaptsda.org/,"[Anderson, Edgar B, Anderson, Victoria C, Berr...",253-589-0766,4.1,"47.143884, -122.560005"
1,Alabama Department of Veterans Affairs,"345 Perry Hill Rd, Montgomery, AL 36109",Montgomery,AL,https://benefits.va.gov/montgomery/,"[Abraham, Shara J, Allen, Andrea N, Allen, Cha...",334-242-5077,3,"32.37564220000001, -86.2455521"
2,American Legion,"3600 Ruddell Rd SE, Lacey, WA 98503",Washington,DC,http://www.walegion.org/,"[Abel, Jami M, Abersold, John L, Abney, John, ...",202-861-2700,4.2,"47.0143149, -122.8082714"
3,American Red Cross,"430 17th St NW, Washington, DC 20006",Washington,DC,http://www.redcross.org/,"[Alberts, Phoebe L, Archie, Latika W, Baker, J...",(202)565-4119,4.2,"38.894852, -77.0403354"
4,American Veterans,"4647 Forbes Blvd, Lanham, MD 20706",Lanham,MD,http://www.amvets.org/,"[Andrews, Gregory C, Bailey, Theo D, Bergmann,...",1-800-810-7148,3.5,"38.9543356, -76.835563"
5,Arizona Department of Veterans Services,"3839 N 3rd St, Phoenix, AZ 85012",Phoenix,AZ,http://www.azdvs.gov/,"[Abraham, Shara J, Alaniz, Heather J, Ambrose,...",602-255-3373,4.6,"33.4927221, -112.0691834"
6,Arkansas Department of Veterans Affairs,"2401 John Ashley Dr, North Little Rock, AR 72114",North Little Rock,AR,http://www.veterans.arkansas.gov/,"[Abraham, Shara J, Aho, Mark R, Alexander, Kat...",501-683-2382,2.2,"34.7787442, -92.289812"
7,Armed Forces Services Corporation,"2800 Shirlington Rd # 350, Arlington, VA 22206",Arlington,VA,https://www.magellanfederal.com/,"[Gaskins, Shaun A, Harlow, Jennifer, Moran, Je...",703-379-9311,4.2,"38.8425, -77.0844444"
8,Blinded Veterans Association,"1600 Duke St #510, Alexandria, VA 22314",Washington,DC,http://www.bva.org/,"[Ainsworth, Wendy I, Allen, Stephanie A, Allfr...",202-371-8880,5,"38.8038229, -77.058415"
9,California Department of Veterans Services,"1227 O St #300, Sacramento, CA 95814",Sacramento,CA,http://www.calvet.ca.gov/,"[Agis, Maria E, Alatorre, Maria L, Albrecht, K...",916-653-2573,3.4,"38.5739622, -121.49188309999998"


In [141]:
df2_grouped.to_csv('orgAttorneyList.csv')